In [1]:
import os
import re
import matplotlib.pyplot as plt
dirname = os.path.join(os.getcwd(), 'train')
imgpath = dirname + os.sep 

images = []
directories = []
dircount = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)

dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  C:\Users\Usuario iTC\Terry\train\
C:\Users\Usuario iTC\Terry\train\CLASS_02 1
C:\Users\Usuario iTC\Terry\train\CLASS_03 62
C:\Users\Usuario iTC\Terry\train\CLASS_04 213
C:\Users\Usuario iTC\Terry\train\CLASS_05 105
C:\Users\Usuario iTC\Terry\train\CLASS_06 949
C:\Users\Usuario iTC\Terry\train\CLASS_07 37
C:\Users\Usuario iTC\Terry\train\CLASS_08 204
Directorios leidos: 7
Imagenes en cada directorio [63, 213, 105, 949, 37, 204, 62]
suma Total de imagenes en subdirs: 1633


In [2]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

Cantidad etiquetas creadas:  1633


In [3]:
carnes=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    carnes.append(name[len(name)-1])
    indice=indice+1

0 CLASS_02
1 CLASS_03
2 CLASS_04
3 CLASS_05
4 CLASS_06
5 CLASS_07
6 CLASS_08


In [4]:
import numpy as np
y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Total number of outputs :  7
Output classes :  [0 1 2 3 4 5 6]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

Training data shape :  (1306, 21, 28, 3) (1306,)
Testing data shape :  (327, 21, 28, 3) (327,)


In [6]:
#plt.figure(figsize=[5,5])

# Display the first image in training data
#plt.subplot(121)
#plt.imshow(train_X[0,:,:], cmap='gray')
#plt.title("Ground Truth : {}".format(train_Y[0]))

# Display the first image in testing data
#plt.subplot(122)
#plt.imshow(test_X[0,:,:], cmap='gray')
#plt.title("Ground Truth : {}".format(test_Y[0]))

In [7]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

In [8]:
import keras
from keras.utils import to_categorical
#from tensorflow.keras.utils import to_categorical
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

Original label: 1
After conversion to one-hot: [0. 1. 0. 0. 0. 0. 0.]


In [9]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

In [10]:
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

(1044, 21, 28, 3) (262, 21, 28, 3) (1044, 7) (262, 7)


In [11]:
#declaramos variables con los parámetros de configuración de la red
INIT_LR = 1e-3 # Valor inicial de learning rate. El valor 1e-3 corresponde con 0.001
epochs = 6 # Cantidad de iteraciones completas al conjunto de imagenes de entrenamiento
batch_size = 100 # cantidad de imágenes que se toman a la vez en memoria

In [12]:
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.activation import LeakyReLU
carnes_model = Sequential()
carnes_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))
carnes_model.add(LeakyReLU(alpha=0.1))
carnes_model.add(MaxPooling2D((2, 2),padding='same'))
carnes_model.add(Dropout(0.5))

carnes_model.add(Flatten())
carnes_model.add(Dense(32, activation='linear'))
carnes_model.add(LeakyReLU(alpha=0.1))
carnes_model.add(Dropout(0.5))
carnes_model.add(Dense(nClasses, activation='softmax'))

In [13]:
carnes_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 21, 28, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 21, 28, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 14, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4928)              0         
                                                                 
 dense (Dense)               (None, 32)                157728    
                                                        

In [14]:
carnes_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(learning_rate=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])

In [15]:
# este paso puede tomar varios minutos, dependiendo de tu ordenador, cpu y memoria ram libre
# como ejemplo, en mi Macbook pro tarda 4 minutos
carnes_train = carnes_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

Epoch 1/6
11/11 [==============================] - 1s 35ms/step - loss: 1.9405 - accuracy: 0.5316 - val_loss: 1.9334 - val_accuracy: 0.5954
Epoch 2/6
11/11 [==============================] - 0s 17ms/step - loss: 1.9305 - accuracy: 0.5805 - val_loss: 1.9244 - val_accuracy: 0.5954
Epoch 3/6
11/11 [==============================] - 0s 17ms/step - loss: 1.9224 - accuracy: 0.5776 - val_loss: 1.9166 - val_accuracy: 0.5954
Epoch 4/6
11/11 [==============================] - 0s 19ms/step - loss: 1.9149 - accuracy: 0.5785 - val_loss: 1.9092 - val_accuracy: 0.5954
Epoch 5/6
11/11 [==============================] - 0s 17ms/step - loss: 1.9096 - accuracy: 0.5785 - val_loss: 1.9025 - val_accuracy: 0.5954
Epoch 6/6
11/11 [==============================] - 0s 17ms/step - loss: 1.9029 - accuracy: 0.5785 - val_loss: 1.8958 - val_accuracy: 0.5954


In [16]:
# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
carnes_model.save("carnes_mnist.h5py")

INFO:tensorflow:Assets written to: carnes_mnist.h5py\assets


INFO:tensorflow:Assets written to: carnes_mnist.h5py\assets


In [17]:
test_eval = carnes_model.evaluate(test_X, test_Y_one_hot, verbose=1)

11/11 [==============================] - 0s 3ms/step - loss: 1.8984 - accuracy: 0.5719


In [18]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 1.8984278440475464
Test accuracy: 0.571865439414978


In [19]:
predicted_classes2 = carnes_model.predict(test_X)

11/11 [==============================] - 0s 3ms/step


In [20]:
predicted_classes=[]
for predicted_carnes in predicted_classes2:
    predicted_classes.append(predicted_carnes.tolist().index(max(predicted_carnes)))
predicted_classes=np.array(predicted_classes)

In [21]:
predicted_classes.shape, test_Y.shape

((327,), (327,))

In [22]:
target_names = ["Class {}".format(i) for i in range(nClasses)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.00      0.00      0.00        12
     Class 1       0.00      0.00      0.00        46
     Class 2       0.00      0.00      0.00        19
     Class 3       0.57      1.00      0.73       187
     Class 4       0.00      0.00      0.00         6
     Class 5       0.00      0.00      0.00        45
     Class 6       0.00      0.00      0.00        12

    accuracy                           0.57       327
   macro avg       0.08      0.14      0.10       327
weighted avg       0.33      0.57      0.42       327



In [23]:
from skimage.transform import resize

images=[]
# AQUI ESPECIFICAMOS UNAS IMAGENES
filenames = ['test/clase3.png']

for filepath in filenames:
    image = plt.imread(filepath,0)
    image_resized = resize(image, (21, 28),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
test_X = X.astype('float32')
test_X = test_X / 255.

predicted_classes = carnes_model.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    print(filenames[i], carnes[img_tagged.tolist().index(max(img_tagged))])

1/1 [==============================] - 0s 15ms/step
test/clase3.png CLASS_05
